In [ ]:
pip install tweepy==4.10.0 ##in case the latest version is not uploaded, necessary for colab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [ ]:
import tweepy 


def authenticate():
  BEARER_TOKEN = 'XXXX' # replace with your bearer token (research access on Twitter API).
  # attempt authentication
  try:
    print("Authenticating...")
    # create Client object
    client = tweepy.Client(bearer_token=BEARER_TOKEN)
    print("API value:....   ", client)
    # return client object
    return client
  except Exception as e:
    print("Error: Authentication Failed", e)

In [ ]:
API = authenticate()


Authenticating...
API value:....    <tweepy.client.Client object at 0x7f3d9f35c070>


In [ ]:
from google.colab import drive #if used on google colab
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#The function that will be used at each iteration to get all tweets matching the request


def get_tweets(query, start_time, end_time, expansions, place_fields, user_fields, tweet_fields, next_token): 

  # empty list to store tweets
  tweets = []
  
  try:
    # call twitter api to collect tweets
    collected_tweets = API.search_all_tweets(query=query,
                         start_time=start_time, 
                         end_time=end_time,
                         expansions=expansions, 
                         place_fields=place_fields,
                         tweet_fields=tweet_fields, 
                         user_fields=user_fields,
                         max_results=500,
                         next_token=next_token
                        )
    
    
  
    return collected_tweets


  except Exception as e:
    print("Error getting tweets", e)

In [ ]:
import pandas as pd
import csv
import time

#Function that specifies that information needed and creates a csv file to store the data 

def append_to_csv(result_set,file_name):

  # A counter variable
  counter = 0
  # Open OR create the target CSV file
  csv_file = open(file_name, "a", newline="", encoding='utf-8')
  csv_writer = csv.writer(csv_file)

  users = result_set.includes["users"]

  users = {user["id"]: user for user in users}


  # Loop through each tweet
  for tweet in result_set.data:
    # We will create a variable for each item since some of the keys might not exist for some tweets
    # So we will account for that

    author_id = tweet['author_id']
    created_at = tweet['created_at']
    if ('geo' in tweet):   
        geo = tweet['geo']
    else:
        geo = " "

    if ('referenced_tweet' in tweet):
      quote_status = tweet['referenced_tweets']['type']
    else:
      quote_status = " "
    

    tweet_id = tweet['id']
    #lang = tweet['lang']
    retweet_count = tweet['public_metrics']['retweet_count']
    reply_count = tweet['public_metrics']['reply_count']
    like_count = tweet['public_metrics']['like_count']
    quote_count = tweet['public_metrics']['quote_count']
    source = tweet['source']
    text = tweet['text']
    created_at = tweet['created_at']
    mentions = tweet['in_reply_to_user_id']


    author = users[tweet.author_id]
    author_username = author.username
    author_name = author.name
    author_created_at = author.created_at
    author_description = author.description
    author_followers_count = author.public_metrics['followers_count']
    author_following_count = author.public_metrics['following_count']
    author_tweet_count = author.public_metrics['tweet_count']
    author_listed_count = author.public_metrics['listed_count']
    author_verified = author.verified
  
    
        
    # Assemble all data in a list
    res = ([mentions,tweet_id, author_id, author_username, author_name, author_created_at, author_description, 
            author_followers_count, author_following_count, author_tweet_count, author_listed_count,
            author_verified, created_at, geo, like_count,
            quote_count, reply_count, retweet_count, quote_status, source, text]) 

    # Append the result to the CSV file
    
    csv_writer.writerow(res)
    counter += 1

  # When done, close the CSV file
  csv_file.close()

  # Print the number of tweets for this iteration
  print("# of Tweets added from this response: ", counter)

In [ ]:
  #Possible data that can be collected with Tweepy
  
  #expansions = ['author_id,in_reply_to_user_id,geo.place_id']
  #tweet_fields = ['attachments','author_id','context_annotations','conversation_id','created_at','edit_controls','entities','geo','id','in_reply_to_user_id','non_public_metrics','organic_metrics','possibly_sensitive','promoted_metrics','public_metrics','referenced_tweets','reply_settings','source','text','withheld']
  #user_fields = ['username', 'public_metrics', 'description', 'location','id','name','pinned_tweet_id','protected','verified','verified_type','withheld']
  #place_fields = ['full_name,id,country,country_code,geo,name,place_type']

In [ ]:
#Function for iterating the data collection 

count = 0 # Counting tweets per time period
#max_count = 1000 # Max tweets per time period
flag = True
#next_token = None
query = '(#balancetonporc OR #metoo OR #metoogay OR #metooinceste OR #iwas) -is:retweet lang:fr'
start_time = '2017-10-01T00:00:01Z'
end_time = '2023-01-01T00:00:01Z'
expansions = ['author_id,geo.place_id,entities.mentions.username,in_reply_to_user_id']
tweet_fields = ['id,text,author_id,geo,created_at,lang,public_metrics,referenced_tweets']
user_fields =  ['id,name,username,created_at,description,public_metrics,verified']
place_fields = ['full_name,id,country,geo']

# Total number of tweets we collected from the loop
# Check if flag is true
def run_collect(flag=True, next_token=None, count=0, total_tweets=0 ):
  if (flag is not True):
    return
  while flag:
    print("-------------------")
    print("Token: ", next_token)
    result = (get_tweets(query=query, 
                         start_time=start_time, 
                         end_time=end_time, 
                         expansions=expansions, 
                         place_fields=place_fields,
                         tweet_fields=tweet_fields, 
                         user_fields=user_fields,
                         next_token=next_token)) # get_tweet function called

    

    if hasattr(result, 'meta'):
      result_count = result.meta['result_count']
      if result_count is not None and result_count > 0:
        #print("Start Date: ", start_list[i])
        append_to_csv(result, "/content/drive/My Drive/data_metoo/base.csv") #specifying the name of the csv file 
        count += result_count
        total_tweets += result_count
        print("Total # of Tweets added: ", total_tweets)
        print("-------------------")
        if result.meta['next_token']:
          # Save the token to use for next call
          next_token = result.meta['next_token']
          print("Next Token: ", next_token)
          time.sleep(5)
          run_collect(True, next_token=next_token, count=count, total_tweets=total_tweets)
        else:
          # Since this is the final request, turn flag to false to move to the next time period.
          flag = False
          next_token = None
    time.sleep(5)
  print("Total number of results: ", total_tweets)


In [ ]:
run_collect()

-------------------
Token:  None
# of Tweets added from this response:  472
Total # of Tweets added:  472
-------------------
Next Token:  b26v89c19zqg8o3fos5u9rzs5yhsrm2d6i1l9kd6r3lvh
-------------------
Token:  b26v89c19zqg8o3fos5u9rzs5yhsrm2d6i1l9kd6r3lvh
# of Tweets added from this response:  477
Total # of Tweets added:  949
-------------------
Next Token:  b26v89c19zqg8o3fo7jcgh99k75rdna1w3mw0qlts5w8t
-------------------
Token:  b26v89c19zqg8o3fo7jcgh99k75rdna1w3mw0qlts5w8t
# of Tweets added from this response:  472
Total # of Tweets added:  1421
-------------------
Next Token:  b26v89c19zqg8o3fo7gdj4ykb3kb9lbcj8mab9h1f9lkt
-------------------
Token:  b26v89c19zqg8o3fo7gdj4ykb3kb9lbcj8mab9h1f9lkt
# of Tweets added from this response:  487
Total # of Tweets added:  1908
-------------------
Next Token:  b26v89c19zqg8o3fo7dgpimmq92kqsjboryvvhtmo8dj1
-------------------
Token:  b26v89c19zqg8o3fo7dgpimmq92kqsjboryvvhtmo8dj1
# of Tweets added from this response:  486
Total # of Tweets 

KeyError: ignored